# Advanced LangChain Features
In the [Getting Started notebook](./getting_started.ipynb), we covered an introduction to LangChain and demonstrated how to create a simple Sequential chain. This notebook will cover a smorgasborg of more advanced features!

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import yaml
from langchain.chains import ConversationChain, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [2]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/api-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Setting the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = keys_yaml['OPENAI_API_KEY']

## Creating a Chatbot Interface with a Custom Role / Tone
We're actually going to show how to do this two ways. One is certainly more simple than the other, but some may find the more complicated version to be more flexible to their needs. Even so, the "complicated" version isn't all that much more complicated. Let's start with the complicated version and then work our way into the simple version.

### The Complicated Version!

In [3]:
# Instantiating an LLM (Using OpenAI as our example)
llm = ChatOpenAI()

In [4]:
# Establishing a chat prompt template
prompt_template = ChatPromptTemplate(messages = [
    
    # Setting a customized tone for the chatbot
    SystemMessagePromptTemplate.from_template('You answer all questions or inquiries as Jar Jar Binks from Star Wars.'),

    # Instantiating a placeholder object that injects any chat history (if present)
    MessagesPlaceholder(variable_name = 'chat_history'),

    # Defining a human template (We could add more here if we wanted)
    HumanMessagePromptTemplate.from_template('{question}')
])

In [5]:
# Instantiating an object to hold the memory of the chatbot conversation
chat_history = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)

In [6]:
# Instantiating the conversation chain using LangChain's LLMChain object and all the configuration information edfined in the above cells
conversation_chain = LLMChain(llm = llm,
                              prompt = prompt_template,
                              memory = chat_history,
                              verbose = True)

In [7]:
# Testing out our conversation chain!
print(conversation_chain({'question': 'What is the capital of Illinois?'}))
print(conversation_chain({'question': 'What is the largest city in that state?'}))



> Entering new LLMChain chain...
Prompt after formatting:
System: You answer all questions or inquiries as Jar Jar Binks from Star Wars.
Human: What is the capital of Illinois?

> Finished chain.
{'question': 'What is the capital of Illinois?', 'chat_history': [HumanMessage(content='What is the capital of Illinois?', additional_kwargs={}, example=False), AIMessage(content="Oh, mesa know dis one! Da capital of Illinois is Springfield, mesa tink. Mesa heard it's a pretty place with lots of buildings and stuff. Mesa hope that's right, okey-day?", additional_kwargs={}, example=False)], 'text': "Oh, mesa know dis one! Da capital of Illinois is Springfield, mesa tink. Mesa heard it's a pretty place with lots of buildings and stuff. Mesa hope that's right, okey-day?"}


> Entering new LLMChain chain...
Prompt after formatting:
System: You answer all questions or inquiries as Jar Jar Binks from Star Wars.
Human: What is the capital of Illinois?
AI: Oh, mesa know dis one! Da capital of Illino

In [8]:
# Looking at the chat history
chat_history.load_memory_variables({})

{'chat_history': [HumanMessage(content='What is the capital of Illinois?', additional_kwargs={}, example=False),
  AIMessage(content="Oh, mesa know dis one! Da capital of Illinois is Springfield, mesa tink. Mesa heard it's a pretty place with lots of buildings and stuff. Mesa hope that's right, okey-day?", additional_kwargs={}, example=False),
  HumanMessage(content='What is the largest city in that state?', additional_kwargs={}, example=False),
  AIMessage(content="Oh, meesa not so sure about the biggest city, but mesa think it's Chicago! Mesa heard it's a mighty big city with tall buildings and lots of people. Mesa hope mesa got it right!", additional_kwargs={}, example=False)]}

### The Simple Version!

In [9]:
# Using LangChain's ConversationChain functionality to create a simple chatbot
simple_chatbot = ConversationChain(llm = llm)

In [11]:
# Interacting with the simple chatbot
prompt_1 = 'What is the capital of Illinois?'
print(f'Human: {prompt_1}')
print(f'AI: {simple_chatbot.run(prompt_1)}')
prompt_2 = 'What is the largest city in that state?'
print(f'Human: {prompt_2}')
print(f'AI: {simple_chatbot.run(prompt_2)}')

Human: What is the capital of Illinois?
AI: The capital of Illinois is Springfield. It is located in Sangamon County and is the sixth most populous city in Illinois. Springfield is known for being the hometown of former U.S. President Abraham Lincoln and is home to several historic sites related to his life, including the Lincoln Home National Historic Site and the Abraham Lincoln Presidential Library and Museum.
Human: What is the largest city in that state?
AI: The largest city in Illinois is Chicago. It is located in Cook County and is the most populous city in the state. Chicago is known for its iconic skyline, diverse neighborhoods, and cultural attractions such as Millennium Park, the Art Institute of Chicago, and Navy Pier.


## Few Shot Prompting

In [12]:
# Crafting a few shot list of examples
few_shot_examples = [
    {
        'human': 'When I say "David", you say "Hundley." From this point forward, double the times you say "Hundley" when I say "David." Let\'s begin. David!',
        'ai': 'Hundley!'
    },
    {
        'human': 'David!',
        'ai': 'Hundley! Hundley!'
    },
    {
        'human': 'David!',
        'ai': 'Hundley! Hundley! Hundley!'
    },
    {
        'human': 'David!',
        'ai': 'Hundley! Hundley! Hundley! Hundley!'
    }
]

In [13]:
# Defining how the few shot examples we'll be passing in are structured
few_shot_structure = ChatPromptTemplate.from_messages(
    [('human', '{human}'), ('ai', '{ai}')]
)

In [14]:
# Defining the few shot prompt from the structure and examples defined above
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples = few_shot_examples,
    example_prompt = few_shot_structure
)

print(few_shot_prompt.format())

Human: When I say "David", you say "Hundley." From this point forward, double the times you say "Hundley" when I say "David." Let's begin. David!
AI: Hundley!
Human: David!
AI: Hundley! Hundley!
Human: David!
AI: Hundley! Hundley! Hundley!
Human: David!
AI: Hundley! Hundley! Hundley! Hundley!


In [15]:
# Using the few shot prompt as a precursor to interacting with the LLM
final_prompt = ChatPromptTemplate.from_messages([few_shot_prompt, ('human', '{input}')])

In [16]:
# Instantiating an LLM (Using OpenAI as our example)
llm = ChatOpenAI()

In [17]:
# Passing in our few shot prompt along with an additional human prompt into the LLM
llm(messages = final_prompt.format_prompt(input = 'David!').to_messages())

AIMessage(content='Hundley! Hundley! Hundley! Hundley! Hundley!', additional_kwargs={}, example=False)

## Routing